In [ ]:
import pandas as pd
import numpy as np

In [ ]:
year=int(snakemake.wildcards.year)
cnt=snakemake.params.countries
act_profile=snakemake.params.act_profile

if act_profile=="EINA 2050 EV":
    seasons=True
else:
    seasons=False

profiles=pd.read_excel(snakemake.input.ev_profiles, engine="calamine",sheet_name=act_profile)
utc=pd.read_csv(snakemake.input.time_zones).set_index('ISO2').loc[cnt,"UTC"]

#dem_shares=pd.read_excel(root+"ev\\country_car_ev_twh.xlsx", engine="calamine", sheet_name="country_car_ev_twh")
country_annual_dem=(pd.pivot_table(
                    (pd.read_csv(snakemake.input.ev_annual_dem)
                    .query("Year==2011 and ISO2.isin(@cnt)")
                    .loc[:,["ISO2","TWh"]]),values="TWh",columns="ISO2")
                    .loc[:,cnt])

#ev_dem_scen="country_car_ev_twh_EINA_"+modelled_yr

In [ ]:
def add_seasons(d):
    
    df_seasons=pd.DataFrame((d.index.month%12 + 3)//3,columns=["season"])
    
    seasons = {
              1: 'Winter',
              2: 'Spring',
              3: 'Summer',
              4: 'Autumn'
              }

    df_seasons['season_name'] = df_seasons['season'].map(seasons)
    
    return d.set_index([d.index,df_seasons["season_name"]])

In [ ]:
dates=pd.date_range(start=pd.Timestamp(year,1,1),end=pd.Timestamp(year,12,31),freq="D")
    
temp=(pd.DataFrame(
            np.zeros((dates.shape[0],len(cnt)))+(1/len(dates)),columns=cnt,index=dates)
            .mul(country_annual_dem.values*1E6,axis=1))
    
if seasons: 
    temp=add_seasons(temp)
else:
    temp["seasons_name"]="ALL_"
    temp=temp.set_index([temp.index,"seasons_name"])

out=[]

for (date,season),d in temp.iterrows():
    
    if date.dayofweek > 4:
        prof=np.tile(profiles[season+"WE"].values.reshape(-1,1),reps=(1,d.shape[0]))
        
    else:
        prof=np.tile(profiles[season+"WD"].values.reshape(-1,1),reps=(1,d.shape[0]))
         
    if date < pd.Timestamp(date.year,3,25) or date > pd.Timestamp(date.year,10,25):
            
        # shift usage profile to align with activity - UTC reference frame
            
        for i,u in enumerate(utc*-1):
            prof[:,i]=np.roll(prof[:,i],u)
                
        out.append(d.values.reshape(1,-1)*prof)
    else:
            
        # shift usage profile to align with daylight savings activity - UTC reference frame
  
        for i,u in enumerate(utc*-1):
            prof[:,i]=np.roll(prof[:,i],u-1)
            
        out.append(d.values.reshape(1,-1)*prof)

In [ ]:
out=pd.DataFrame(np.concatenate(out,axis=0),
                     index=pd.date_range(start=temp.index[0][0],end=temp.index[-1][0]+pd.Timedelta(hours=23),freq="1h"),
                     columns=temp.columns)

In [ ]:
out.to_csv(snakemake.output[0])